In [1]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")

In [2]:
import torch

In [3]:
from nnetworks.layers.config import LstmConfig, DenseInput, MaskedConv1DConfig, ResBlockConfig, TransformerEncodersBlockConfig
from nnetworks.models.config import MuNuSepLstmConfig, MuNuSepResNetConfig
from nnetworks.models.munusep_lstm import MuNuSepLstm
from nnetworks.models.munusep_resnet import MuNuSepResNet

from nnetworks.models.config_manager import save_model_cfg, model_from_yaml

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Attention based model

In [5]:
from nnetworks.models.munusep_transformer import TransformerClassifier
from nnetworks.layers.config import DenseInput, TransformerEncodersBlockConfig
from nnetworks.models.config import DenseInput, TransformerClassifierConfig

In [22]:
nhead = 4
d_model = 256
dim_feedforward = 256
cfg = TransformerClassifierConfig(
        encoder_config=TransformerEncodersBlockConfig(
            in_features = 5,
            encoders_number = 2,
            d_model = d_model,
            nhead = nhead,
            dim_feedforward = dim_feedforward,
            dropout = 0.1,
            layer_norm_eps = 0.00001,
            activation = {"LeakyReLU": {"negative_slope": 0.1}}
        ),
        pooling_type="Average",
        dense_layers=[
            DenseInput(
                d_model*2, # *2 beacuse of concat with classification token 
                32,
                activation = {'LeakyReLU': dict(negative_slope=0.1)},
                dropout = 0.1,
                do_norm = True
                ),
            
            DenseInput(
                32, 
                2,
                activation = {'Softmax': {"dim": 1}},
                dropout = 0.,
                do_norm = False
                ),
        ]
    )

model = TransformerClassifier(cfg)
count_parameters(model)

809609

In [18]:
# test
data = torch.rand((2,9,5))
mask = torch.ones((2,9,1))
mask[:,7:,:] = 0.
model.eval()
model(data, mask=mask), model(data*mask, mask=mask), model(data[:,:7], mask=mask[:,:7])

(tensor([[0.3897, 0.6103],
         [0.3921, 0.6079]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.3897, 0.6103],
         [0.3921, 0.6079]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.3897, 0.6103],
         [0.3921, 0.6079]], grad_fn=<SoftmaxBackward0>))

In [10]:
name_for_model_arch = "munusep_all_transformer"
# save_model_cfg(cfg, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

In [11]:
loaded_model = model_from_yaml(TransformerClassifier, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")
count_parameters(loaded_model)

108681

## Test gpu memory usage

In [21]:
device = torch.device("cuda:0")
loaded_model.to(device)
_ = loaded_model(torch.rand((512,100,5)).to(device), torch.ones((512,100,1)).to(device))

# LSTM based model

In [ ]:
cfg = MuNuSepLstmConfig(
        lstm_layers=[
            LstmConfig(
                5,
                64,
                1,
                return_sequences=True,
                dropout = 0.,
                bidirectional=True,
                norm = {"MaskedBatchNorm1d": None}
            )
        ],
        pooling_type="Average",
        dense_layers=[
            DenseInput(
                (64)*2, 
                32,
                activation = {'LeakyReLU': dict(negative_slope=0.1)},
                dropout = 0.1,
                do_norm = True
                ),
            
            DenseInput(
                32, 
                2,
                activation = {'Softmax': {"dim": 1}},
                dropout = 0.,
                do_norm = False
                ),
        ]
    )

model = MuNuSepLstm(cfg)
print(count_parameters(model))

# test

model(torch.rand(2,10,5))

40866


tensor([[0.4440, 0.5560],
        [0.5806, 0.4194]], grad_fn=<SoftmaxBackward0>)

In [18]:
name_for_model_arch = "munusep_all_rnn"
save_model_cfg(cfg, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

In [19]:
model_from_yaml(MuNuSepLstm, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

MuNuSepLstm(
  (lstm_layers): ModuleList(
    (0): LstmLayer(
      (lstm_layer): LSTM(5, 64, batch_first=True, bidirectional=True)
      (norm_layer): MaskedLayerNorm12()
    )
  )
  (pooling): GlobalAveragePooling1DMasked()
  (dense_layers): ModuleList(
    (0): DenseBlock(
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (dense_layer): Linear(in_features=128, out_features=32, bias=True)
      (activation): LeakyReLU(negative_slope=0.1)
      (norm1d): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): DenseBlock(
      (dropout_layer): Dropout(p=0.0, inplace=False)
      (dense_layer): Linear(in_features=32, out_features=2, bias=True)
      (activation): Softmax(dim=1)
    )
  )
)

# ResNet Based model

In [4]:
# Tiny ResNet
cfg = MuNuSepResNetConfig(
        res_blocks = [
            ResBlockConfig(
                id = MaskedConv1DConfig(
                        in_channels = 5,
                        out_channels = 64,
                        kernel_size = 3,
                        strides = 1,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                    ),
                cd = MaskedConv1DConfig(
                        in_channels = 64,
                        out_channels = 128,
                        kernel_size = 3,
                        strides = 2,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                    ),
                skip = MaskedConv1DConfig(
                        in_channels = 5,
                        out_channels = 128,
                        kernel_size = 5,
                        strides = 2,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                )
            )
        ],
        pooling_type = "Average",
        dense_layers = [
            DenseInput(
                in_features = 256, 
                units = 32,
                activation = {'ReLU': None},
                dropout = 0.1,
                do_norm = True
                ),
            DenseInput(
                in_features = 32, 
                units = 2,
                activation = {'Softmax': {"dim": 1}},
                do_norm = False,
                dropout = 0.
                ),
        ]
    )

model = MuNuSepResNet(cfg)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

# test
model(torch.rand((3,10, 5)), torch.ones((3,10,1)))

38050


tensor([[0.7440, 0.2560],
        [0.3371, 0.6629],
        [0.3133, 0.6867]], grad_fn=<SoftmaxBackward0>)

In [11]:
name_for_model_arch = "munusep_all_resnet"
#save_model_cfg(cfg, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

In [13]:
loaded_model = model_from_yaml(MuNuSepResNet, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")
count_parameters(loaded_model)

637090

In [14]:
device = torch.device("cuda:0")
loaded_model.to(device)
_ = loaded_model(torch.rand((512,100,5)).to(device), torch.ones((512,100,1)).to(device))

In [20]:
import numpy as np

In [24]:
np.arange(10)[(torch.ones(10)==1).numpy()]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])